In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
transitions = spark.read.table("workspace.salesforce.lead_status_transitions").\
filter("Field = 'Status'").\
select("OldValue", "NewValue", "total").withColumnRenamed("CreatedDate", "created_at")
display(transitions)

## Markov Chain Model - Transition Probability Matrix

In [0]:
# Count transitions from each current_status → next_status
transition_counts = (
    transitions
    .groupBy("OldValue", "NewValue")
    .agg(F.sum("total").alias("count"))
)

# Total transitions from each current_status
state_totals = (
    transition_counts
    .groupBy("OldValue")
    .agg(F.sum("count").alias("total"))
)

# Compute probabilities
transition_probs = (
    transition_counts
    .join(state_totals, "OldValue")
    .withColumn("probability", F.col("count") / F.col("total"))
    .select(
        F.col("OldValue").alias("from_status"),
        F.col("NewValue").alias("to_status"),
        "count",
        F.round("probability", 4).alias("probability")
    )
    .orderBy("from_status", F.desc("probability"))
)

display(transition_probs)

## Visualize Transition Probability Matrix


In [0]:
# Convert to pandas for visualization
prob_matrix_df = transition_probs.toPandas()

# Create pivot table for heatmap
pivot = prob_matrix_df.pivot(
    index='from_status', 
    columns='to_status', 
    values='probability'
).fillna(0)

# Create heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(
    pivot, 
    annot=True, 
    fmt='.3f', 
    cmap='YlOrRd',
    cbar_kws={'label': 'Transition Probability'},
    linewidths=0.5
)
plt.title('Lead Status Transition Probability Matrix (Markov Chain)', fontsize=16, pad=20)
plt.xlabel('To Status', fontsize=12)
plt.ylabel('From Status', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
import os 
# Save figure to same directory as notebook/script
output_path = os.path.join(os.getcwd(), "lead_status_transition_heatmap.png")
plt.savefig(output_path, dpi=300, bbox_inches='tight')

display(plt.show())

### ✅ **1. Re-engagement Success Rate**

**Current:** 37.5% of re-engaged leads become *Qualified*
**Insight:** The re-engagement campaigns are effective — they revive dormant leads.
**Actionable Metric:**

> **Goal:** Increase *Re-engagement → Qualified* rate from **37.5% → 50%** by improving email personalization and timing.

---

### ⚠️ **2. Contacted Drop-off Rate**

**Current:** 100% of *Contacted* leads become *Unqualified*
**Insight:** Major breakdown after initial outreach — leads are not converting after contact.
**Actionable Metric:**

> **Goal:** Reduce *Contacted → Unqualified* rate from **100% → <50%** by refining outreach scripts and follow-up cadence.

---

### 🔴 **3. Qualified Reversal Rate**

**Current:** 100% of *Qualified* leads later marked *Unqualified*
**Insight:** Qualification criteria are too loose — leads pass the filter but later get rejected.
**Actionable Metric:**

> **Goal:** Lower *Qualified → Unqualified* rate from **100% → <20%** by tightening qualification rules or introducing a scoring threshold.